In [7]:
import cv2
import numpy as np
DATA_PATH = 'C:/Users/amityu/Gel_Sheet_Data/'
#movie = 'Control'
#movie = '130721'
#movie ='140721'
#movie ='150721'
#movie ='100621'
movie = '130721_CCA60_RAW'
MOVIE_PATH = DATA_PATH +  movie + '/'
GRAPH_PATH = 'C:/Users/amityu/Gel_Sheet_Graph/'


In [8]:
imgs_float = np.load(MOVIE_PATH + 'np/gel.npy')[:,:,:,256] # replace with your images
imgs = np.zeros(imgs_float.shape, dtype='uint8')
imgs_float = np.nan_to_num(imgs_float)
for t in range(len(imgs_float)):
    img = imgs_float[t]
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)
    img = img.astype(np.uint8)
    imgs[t] = img

#replace nan with zero

In [ ]:
import cv2
import numpy as np

# Global variables
ix, iy = -1, -1
drawing = False
img_index = 0
r_size = 40 # Define your rectangle size here
coordinates = []

# Define the draw_rectangle callback function
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, img, img_index, r_size, img_orig

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
        img = img_orig.copy()  # create a copy of the image

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing == True:
            img_temp = img.copy()
            cv2.rectangle(img_temp, pt1=(ix, iy), pt2=(ix+r_size, iy+r_size), color=(0, 255, 0), thickness=2)
            cv2.imshow('image', img_temp)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img, pt1=(ix, iy), pt2=(ix+r_size, iy+r_size), color= (255,255,255), thickness=2)
        coordinates.append((img_index, ix, iy))
        print(f'Saved coordinates: {coordinates[-1]}')

    elif event == cv2.EVENT_MOUSEWHEEL:
        if flags > 0:  # scroll up
            img_index = (img_index + 1) % len(imgs)  # cycle through images
        else:  # scroll down
            img_index = (img_index - 1) % len(imgs)  # cycle through images
        img_orig = imgs[img_index].copy()  # load the new image
        img = img_orig.copy()
        cv2.imshow('image', img)


# 3D array of images
img = imgs[img_index].copy()

# Create the named window and set the mouse callback
cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_rectangle)

# Show the image and wait for the user to close the window
while True:
    cv2.imshow('image', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

# Print the saved coordinates
print('All saved coordinates:')
for idx, x, y in coordinates:
    print(f'Image index: {idx}, X: {x}, Y: {y}')


Saved coordinates: (57, 271, 41)
Saved coordinates: (57, 350, 58)
